In [14]:
import pandas as pd
import os, random, glob, json
from sentence_transformers import SentenceTransformer
import nltk
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [5]:
files = glob.glob('en/en/mpqa/tab_feature_files/*.feat')
empty_list = []
final_list = []
header_names = ["sentence_id", "token_id", "token", "lemma", "pos", "term_id", "pol/mod", "poldomain", "aspect_training", "entity", "property", "phrase_type", "sentiment" , ]

for file in files:
    #print(file)
    frame= pd.read_csv(file,sep="\t", header=None, names=header_names)
    list_of_tokens = []
    list_of_tokens.append(frame['token'].tolist())
    

    for a in list_of_tokens:
        #string_tokens = str(list_of_tokens)
        string_per_file = " ".join(a)
        #print(string_per_file)
        break
        

    list_of_sentiment = []
    list_of_sentiment.append(frame['sentiment'].tolist())
    #print(list_of_sentiment)

    score = 0
    for a in list_of_sentiment:
        for b in a:
            if b == "B-negative":
                score += -1
            if b == "B-positive":
                score += 1
    #print(score)
    sentiment = ""
    if score > 0:
        sentiment = "positive"
    elif score == 0:
        sentiment = "neutral"
    else:
        sentiment = "negative"
    #print(sentiment)
    sentiment_dict = dict()
    sentiment_dict[string_per_file] = sentiment
    empty_list.append(string_per_file)
    
    #print(sentiment_dict)
    final_list.append(sentiment_dict)
#print(final_list)    
    #turn into dataframe

print(len(files))

506


In [7]:
final_list[0]
text_list = []
label_list = []
for d in final_list:
    for key, value in d.items():
        text_list.append(key)
        label_list.append(value)
        
data_text={'text':text_list, 'label': label_list}
df=pd.DataFrame(data=data_text)
df

,text,label
0,the Kimberley Provincial Hospital say it would...,negative
1,Beijing have complete the afforestation of @ca...,positive
2,Russian guard seize @card@ kg of heroin on Taj...,neutral
3,"Istanbul , Dec @card@ ( a.a ) - Rauf Denktas ,...",positive
4,"Tehran , Dec @card@ IRNA -- President Muhammad...",neutral
5,the Sulaymaniyah-based telecommunication compa...,neutral
6,Jakarta ( Agency ) : Indonesia will not follow...,neutral
7,"Tokyo , Dec. @card@ ( Yonhap ) -- Japanese def...",neutral
8,France 's Chirac talk to Pakistan 's Musharraf...,negative
9,"the Argentine TV station Todo Noticias ["" all ...",negative


In [20]:
model = SentenceTransformer('roberta-large-nli-mean-tokens')
texts = df.text.to_list()

embedded_texts=[]
for text in texts:
    sentences = nltk.sent_tokenize(text)
    if len(sentences) >=6:
        sentence_embeddings = model.encode(sentences[:6])
    else:
        for n in range (6-len(sentences)):
            sentences.append(sentences[0])
        sentence_embeddings = model.encode(sentences[:6])
    embedded_texts.append(sentence_embeddings)

I0317 16:41:30.072726 11152 SentenceTransformer.py:29] Load pretrained SentenceTransformer: roberta-large-nli-mean-tokens
I0317 16:41:30.092728 11152 SentenceTransformer.py:32] Did not find a / or \ in the name. Assume to download model from server
I0317 16:41:30.104728 11152 SentenceTransformer.py:54] Downloading sentence transformer model from https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/roberta-large-nli-mean-tokens.zip and saving it at C:\Users\ApotheekStiens/.cache\torch\sentence_transformers\public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_roberta-large-nli-mean-tokens.zip

  0%|                                              | 0.00/1.31G [00:00<?, ?B/s]
  0%|                                      | 81.9k/1.31G [00:00<26:41, 819kB/s]
  0%|                                       | 213k/1.31G [00:00<23:41, 923kB/s]
  0%|                                      | 426k/1.31G [00:00<20:01, 1.09MB/s]
  0%|                            

  6%|██▏                                  | 75.5M/1.31G [00:10<03:25, 6.03MB/s]
  6%|██▏                                  | 76.1M/1.31G [00:10<03:26, 5.98MB/s]
  6%|██▏                                  | 76.8M/1.31G [00:10<03:20, 6.15MB/s]
  6%|██▏                                  | 77.4M/1.31G [00:10<03:22, 6.10MB/s]
  6%|██▏                                  | 78.1M/1.31G [00:10<03:29, 5.90MB/s]
  6%|██▏                                  | 78.6M/1.31G [00:10<03:42, 5.55MB/s]
  6%|██▏                                  | 79.3M/1.31G [00:10<03:29, 5.88MB/s]
  6%|██▎                                  | 80.0M/1.31G [00:10<03:28, 5.91MB/s]
  6%|██▎                                  | 80.7M/1.31G [00:10<03:18, 6.19MB/s]
  6%|██▎                                  | 81.3M/1.31G [00:11<03:16, 6.26MB/s]
  6%|██▎                                  | 82.0M/1.31G [00:11<03:13, 6.36MB/s]
  6%|██▎                                  | 82.7M/1.31G [00:11<03:08, 6.51MB/s]
  6%|██▎                                

 11%|████▏                                 | 143M/1.31G [00:21<02:42, 7.21MB/s]
 11%|████▏                                 | 143M/1.31G [00:21<02:41, 7.22MB/s]
 11%|████▏                                 | 144M/1.31G [00:21<02:45, 7.07MB/s]
 11%|████▏                                 | 145M/1.31G [00:21<02:40, 7.27MB/s]
 11%|████▏                                 | 146M/1.31G [00:21<02:41, 7.24MB/s]
 11%|████▏                                 | 146M/1.31G [00:21<02:37, 7.38MB/s]
 11%|████▎                                 | 147M/1.31G [00:21<02:36, 7.47MB/s]
 11%|████▎                                 | 148M/1.31G [00:21<02:31, 7.67MB/s]
 11%|████▎                                 | 149M/1.31G [00:21<02:34, 7.54MB/s]
 11%|████▎                                 | 150M/1.31G [00:21<02:31, 7.67MB/s]
 11%|████▎                                 | 150M/1.31G [00:22<02:29, 7.77MB/s]
 12%|████▍                                 | 151M/1.31G [00:22<02:30, 7.73MB/s]
 12%|████▍                              

 16%|██████▏                               | 213M/1.31G [00:32<03:15, 5.62MB/s]
 16%|██████▏                               | 213M/1.31G [00:32<03:26, 5.32MB/s]
 16%|██████▏                               | 214M/1.31G [00:32<03:48, 4.80MB/s]
 16%|██████▏                               | 214M/1.31G [00:32<03:42, 4.94MB/s]
 16%|██████▏                               | 215M/1.31G [00:32<03:51, 4.75MB/s]
 16%|██████▏                               | 215M/1.31G [00:32<04:11, 4.36MB/s]
 16%|██████▎                               | 216M/1.31G [00:32<03:53, 4.69MB/s]
 16%|██████▎                               | 216M/1.31G [00:33<03:55, 4.65MB/s]
 17%|██████▎                               | 217M/1.31G [00:33<03:49, 4.78MB/s]
 17%|██████▎                               | 217M/1.31G [00:33<03:43, 4.90MB/s]
 17%|██████▎                               | 218M/1.31G [00:33<03:43, 4.89MB/s]
 17%|██████▎                               | 218M/1.31G [00:33<03:36, 5.06MB/s]
 17%|██████▎                            

 21%|███████▊                              | 272M/1.31G [00:43<03:31, 4.91MB/s]
 21%|███████▉                              | 272M/1.31G [00:43<03:31, 4.91MB/s]
 21%|███████▉                              | 273M/1.31G [00:43<03:43, 4.65MB/s]
 21%|███████▉                              | 273M/1.31G [00:43<03:42, 4.67MB/s]
 21%|███████▉                              | 274M/1.31G [00:43<03:40, 4.71MB/s]
 21%|███████▉                              | 274M/1.31G [00:43<03:51, 4.49MB/s]
 21%|███████▉                              | 275M/1.31G [00:43<03:50, 4.50MB/s]
 21%|███████▉                              | 275M/1.31G [00:44<03:45, 4.60MB/s]
 21%|███████▉                              | 276M/1.31G [00:44<03:55, 4.40MB/s]
 21%|███████▉                              | 276M/1.31G [00:44<04:05, 4.21MB/s]
 21%|████████                              | 276M/1.31G [00:44<04:06, 4.21MB/s]
 21%|████████                              | 277M/1.31G [00:44<04:11, 4.12MB/s]
 21%|████████                           

...

 88%|████████████████████████████████▌    | 1.16G/1.31G [03:21<00:42, 3.64MB/s]
 88%|████████████████████████████████▌    | 1.16G/1.31G [03:21<00:45, 3.42MB/s]
 88%|████████████████████████████████▌    | 1.16G/1.31G [03:21<00:45, 3.43MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:21<00:44, 3.48MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:21<00:51, 2.98MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<00:52, 2.96MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<01:03, 2.41MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<01:00, 2.52MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<00:56, 2.74MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<00:53, 2.87MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<00:47, 3.18MB/s]
 88%|████████████████████████████████▋    | 1.16G/1.31G [03:22<00:46, 3.27MB/s]
 88%|████████████████████████████████▋  

 92%|██████████████████████████████████   | 1.21G/1.31G [03:33<00:24, 4.27MB/s]
 92%|██████████████████████████████████   | 1.21G/1.31G [03:33<00:26, 3.85MB/s]
 92%|██████████████████████████████████   | 1.21G/1.31G [03:33<00:26, 3.84MB/s]
 92%|██████████████████████████████████   | 1.21G/1.31G [03:34<00:28, 3.59MB/s]
 92%|██████████████████████████████████   | 1.21G/1.31G [03:34<00:28, 3.64MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:24, 4.07MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:22, 4.53MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:23, 4.28MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:23, 4.27MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:23, 4.22MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:22, 4.40MB/s]
 92%|██████████████████████████████████▏  | 1.21G/1.31G [03:34<00:24, 4.08MB/s]
 93%|██████████████████████████████████▏

 96%|███████████████████████████████████▎ | 1.25G/1.31G [03:46<00:39, 1.47MB/s]
 96%|███████████████████████████████████▎ | 1.25G/1.31G [03:46<00:37, 1.56MB/s]
 96%|███████████████████████████████████▍ | 1.25G/1.31G [03:46<00:40, 1.42MB/s]
 96%|███████████████████████████████████▍ | 1.25G/1.31G [03:46<00:37, 1.54MB/s]
 96%|███████████████████████████████████▍ | 1.25G/1.31G [03:46<00:33, 1.69MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:46<00:37, 1.51MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:46<00:35, 1.59MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:46<00:35, 1.61MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:47<00:32, 1.72MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:47<00:30, 1.82MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:47<00:31, 1.78MB/s]
 96%|███████████████████████████████████▍ | 1.26G/1.31G [03:47<00:30, 1.81MB/s]
 96%|███████████████████████████████████

 98%|████████████████████████████████████▎| 1.29G/1.31G [03:57<00:07, 3.20MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:07, 3.17MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:07, 3.46MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:07, 3.40MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:05, 4.02MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:05, 3.98MB/s]
 98%|████████████████████████████████████▎| 1.29G/1.31G [03:58<00:05, 4.09MB/s]
 98%|████████████████████████████████████▍| 1.29G/1.31G [03:58<00:05, 4.31MB/s]
 98%|████████████████████████████████████▍| 1.29G/1.31G [03:58<00:05, 3.97MB/s]
 98%|████████████████████████████████████▍| 1.29G/1.31G [03:58<00:05, 4.14MB/s]
 98%|████████████████████████████████████▍| 1.29G/1.31G [03:59<00:04, 4.37MB/s]
 98%|████████████████████████████████████▍| 1.29G/1.31G [03:59<00:04, 4.60MB/s]
 99%|███████████████████████████████████

OSError: [Errno 28] No space left on device

In [21]:
# use concatenated list or mean list in the train_test_split function below

concatenated_list = []
mean_list=[]

for six_embeds in embedded_texts:
    new = np.concatenate(six_embeds)
    concatenated_list.append(new)
    mean_embeds_per_text = np.mean(six_embeds, axis = 0)
    mean_list.append(mean_embeds_per_text)

In [22]:
training_labels = list(df.label)

In [23]:
from sklearn.model_selection import train_test_split
# mean_list can be exchanged for concatenated_list
X_train, X_test, y_train, y_test = train_test_split(concatenated_list, training_labels, test_size=0.33, random_state=42)

In [24]:
rfc = RandomForestClassifier(n_estimators=10)
predictionsRF=rfc.fit(X_train, y_train).predict(X_test)

gnb = GaussianNB()
predictionsNB = gnb.fit(X_train, y_train).predict(X_test)

svm_var = LinearSVC(random_state=0, tol=1e-5, max_iter = 10000)
svm_var.fit(X_train, y_train)
predictionsSVM = svm_var.predict(X_test)

clf = MLPClassifier(solver='sgd', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1, max_iter=500)
clf.fit(X_train, y_train)
predictionsMLP=clf.predict(X_test)

clf = MLPClassifier(solver='sgd', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1, max_iter=500)
clf.fit(X_train, y_train)
predictionsMLPsgd=clf.predict(X_test)

clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1, max_iter=500)
clf.fit(X_train, y_train)
predictionsMLPadam=clf.predict(X_test)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(15,), random_state=1, max_iter=500)
clf.fit(X_train, y_train)
predictionsMLPlbfgs=clf.predict(X_test)

In [25]:
print("Naive Bayes (Gaussian):")
print(classification_report(y_test, predictionsNB))
print("Random Forest:")
print(classification_report(y_test, predictionsRF))
print("Support Vector Machine:")
print(classification_report(y_test, predictionsSVM))
print("Multi layer perceptron (stochastic gradient descent):")
print(classification_report(y_test, predictionsMLPsgd))
print("Multi layer perceptron (adam):")
print(classification_report(y_test, predictionsMLPadam))
print("Multi layer perceptron (lbfgs):")
print(classification_report(y_test, predictionsMLPlbfgs))



Naive Bayes (Gaussian):
              precision    recall  f1-score   support

    negative       0.64      0.71      0.68        63
     neutral       0.57      0.56      0.56        52
    positive       0.50      0.44      0.47        52

   micro avg       0.58      0.58      0.58       167
   macro avg       0.57      0.57      0.57       167
weighted avg       0.58      0.58      0.58       167

Random Forest:
              precision    recall  f1-score   support

    negative       0.56      0.76      0.64        63
     neutral       0.52      0.52      0.52        52
    positive       0.52      0.29      0.37        52

   micro avg       0.54      0.54      0.54       167
   macro avg       0.53      0.52      0.51       167
weighted avg       0.53      0.54      0.52       167

Support Vector Machine:
              precision    recall  f1-score   support

    negative       0.62      0.73      0.67        63
     neutral       0.64      0.65      0.65        52
    positive